# SFC Embedding Example

Solve the example using library `pulp`

Documentation: https://coin-or.github.io/pulp/


<img src="https://uphinh.vn/images/2022/04/12/54aa334d32010c9f193471fd7d9bfbc1.md.png" alt="drawing" width="300"/>

In [2]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 12.8 MB/s 


In [3]:
from pulp import *

# Network setup
Ns, Nv, Es, Ev = 3, 2, 6, 1
Es_id = [12, 13, 21, 23, 31, 32]
Ev_id = [12]
ids = {
  "x": [[1,1],[1,2],[1,3],[2,1],[2,2],[2,3]],
  "y": [[1,2,1,2],[1,2,1,3],[1,2,2,1],[1,2,2,3],[1,2,3,1],[1,2,3,2]]
}

# Create the model
model = LpProblem(name="embedding", sense=LpMinimize)

# Initialize the decision variables: x and y are binaries
x = LpVariable.dicts(name="x", indices=[(v+1, i+1) 
                    for v in range(Nv) for i in range(Ns)], cat='Binary')
y = LpVariable.dicts(name="y", indices=[(Ev_id[ev], Es_id[es]) 
                    for ev in range(Ev) for es in range(Es)], cat='Binary')

#### **Constraint 1:** Capacity limit constraint
$$
\begin{align*}
\sum_{v\in\mathcal{N}_{s}}r_{v} x_{i}^{v} & \leq a_{i},\quad\forall i\in\mathcal{N},\\
\sum_{vw\in\mathcal{E}_{s}}r_{vw} y_{ij}^{vw} & \leq a_{ij},\quad\forall ij\in\mathcal{E}.
\end{align*}
$$

In [4]:
model += (2* x[(1,1)] + 2* x[(2,1)] <= 10, "C1_Node_1")
model += (2* x[(1,2)] + 2* x[(2,2)] <= 10, "C1_Node_2")
model += (2* x[(1,3)] + 2* x[(2,3)] <= 10, "C1_Node_3")

model += (2* y[(12,12)] <= 10, "C1_Link_1")
model += (2* y[(12,13)] <= 10, "C1_Link_2")
model += (2* y[(12,21)] <= 10, "C1_Link_3")
model += (2* y[(12,23)] <= 10, "C1_Link_4")
model += (2* y[(12,31)] <= 10, "C1_Link_5")
model += (2* y[(12,32)] <= 10, "C1_Link_6")

#### **Constraint 2:** Each VNF in the SFC is mapped onto *different* physical node
$$
\begin{align*}
\sum_{v\in\mathcal{N}_{s}} x_{i}^{v} \leq1,\quad\forall i\in\mathcal{N}.
\end{align*}
$$

In [5]:
model += (x[(1,1)] + x[(2,1)] <= 1, "C2_1")
model += (x[(1,2)] + x[(2,2)] <= 1, "C2_2")
model += (x[(1,3)] + x[(2,3)] <= 1, "C2_3")

#### **Constraint 3:** Each VNF in the SFC is mapped to *only* one physical node
$$
\begin{align*}
\sum_{i\in\mathcal{N}} x_{i}^{v} = 1,\quad\forall v\in\mathcal{N}_{s}.
\end{align*}
$$



In [6]:
model += (x[(1,1)] + x[(1,2)] + x[(1,3)] == 1, "C3_1")
model += (x[(2,1)] + x[(2,2)] + x[(2,3)] == 1, "C3_2")

#### **Constraint 4:** Flow conservation
$$
\begin{align*}
\sum_{j\in\mathcal{N}} y_{ij}^{vw}-\sum_{j\in\mathcal{N}} y_{ji}^{vw} = x_{i}^{v} - x_{i}^{w}, \quad\forall i\in\mathcal{N},vw\in\mathcal{E}_{s}.
\end{align*}
$$


In [7]:
model += (x[(1,1)] - x[(2,1)] - y[(12,12)] - y[(12,13)] + y[(12,21)] + y[(12,31)] == 0, "C4_1")
model += (x[(1,2)] - x[(2,2)] + y[(12,12)] - y[(12,21)] - y[(12,23)] + y[(12,32)] == 0, "C4_2")
model += (x[(1,3)] - x[(2,3)] + y[(12,13)] + y[(12,23)] - y[(12,31)] - y[(12,32)] == 0, "C4_3")

#### **Objective function:** 
$$
\min \underbrace{\sum_{i\in\mathcal{N}}\sum_{v\in\mathcal{N}_{s}}r_{v}c_{i}x_{i}^{v}}_{\mbox{node resource cost}}+\underbrace{\sum_{ij\in\mathcal{E}}\sum_{vw\in\mathcal{E}_{s}}r_{vw}c_{ij}y_{ij}^{vw}}_{\mbox{link resource cost}}+\underbrace{\sum_{ij\in\mathcal{E}}\sum_{vw\in\mathcal{E}_{s}}d_{ij}y_{ij}^{vw}}_{\mbox{latency}}
$$

In [8]:
model +=  2* x[(1,1)] + 2* x[(2,1)] + 2* x[(1,2)] + \
          2* x[(2,2)] + 2* x[(1,3)] + 2* x[(2,3)] + \
          4* y[(12,12)] + 5* y[(12,13)] + 4* y[(12,21)] + \
          3* y[(12,23)] + 5* y[(12,31)] + 3* y[(12,32)]

### Solve the optimization problem

In [9]:
status = model.solve()

print(f"Status: {model.status}, {LpStatus[model.status]}")
print(f"\nObjective: {model.objective.value()}")
print('\nVariables:')
for var in model.variables():
    print(f"\t{var.name}: \t {var.value()}")

# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}")

# Mapped nodes and links
print('\nEmbedding solution:')
print('Nodes:')
for i in range(len(x)):
  tmp = ids['x'][i]
  m = tmp[0]
  n = tmp[1]
  xi = x[(m,n)].value()
  if xi == 1:
    print('\t(v' + str(m) + ') -> (i' + str(n) + ')')

print('Links:')    
for i in range(len(y)):
  tmp = ids['y'][i]
  m = int(str(tmp[0]) + str(tmp[1]))
  n = int(str(tmp[2]) + str(tmp[3]))
  yi = y[(m, n)].value()
  if yi == 1:
    print('\t(v' + str(tmp[0]) + '-v' + str(tmp[1]) + ') -> (i' + str(tmp[2]) + '-i' + str(tmp[3]) + ')')


Status: 1, Optimal

Objective: 7.0

Variables:
	x_(1,_1): 	 0.0
	x_(1,_2): 	 1.0
	x_(1,_3): 	 0.0
	x_(2,_1): 	 0.0
	x_(2,_2): 	 0.0
	x_(2,_3): 	 1.0
	y_(12,_12): 	 0.0
	y_(12,_13): 	 0.0
	y_(12,_21): 	 0.0
	y_(12,_23): 	 1.0
	y_(12,_31): 	 0.0
	y_(12,_32): 	 0.0

Embedding solution:
Nodes:
	(v1) -> (i2)
	(v2) -> (i3)
Links:
	(v1-v2) -> (i2-i3)
